In [1]:
# !pip install sklearn torch datasets transformers

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
# cd /content/drive/MyDrive/research/code/toxic-comments/Scripts/

In [4]:
import pandas as pd

dataset_number = 1

# Lacafe
# text = 'txt'
# label = 'has_anger'
# train_data = pd.read_csv(f'./Datasets/Lacafe/df_dataset_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/Lacafe/df_dataset_test_{dataset_number}.csv')

# Fortuna
# text = 'text'
# label = 'hatespeech_comb'
# train_data = pd.read_csv(f'./Datasets/Fortuna/2019-05-28_portuguese_hate_speech_binary_classification_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/Fortuna/2019-05-28_portuguese_hate_speech_binary_classification_test_{dataset_number}.csv')

# ToLD-BR
# text = 'text'
# label = 'hatespeech'
# train_data = pd.read_csv(f'./Datasets/ToldBR/ToLD-BR-Treated_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/ToldBR/ToLD-BR-Treated_test_{dataset_number}.csv')

# ToLD-BR (undersampling)
text = 'text'
label = 'hatespeech'
train_data = pd.read_csv(f'./Datasets/ToldBRUndersampling/ToLD-BR-Treated_undersampling_train_{dataset_number}.csv')
test_data = pd.read_csv(f'./Datasets/ToldBRUndersampling/ToLD-BR-Treated_undersampling_test_{dataset_number}.csv')

# OffComBr
# text = 'text'
# label = 'offensive'
# train_data = pd.read_csv(f'./Datasets/OffComBR/OffComBR2_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/OffComBR/OffComBR2_test_{dataset_number}.csv')

In [5]:
from datasets import Dataset, DatasetDict

print(train_data)
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

raw_datasets = DatasetDict({'train': train_data, 'test': test_data})

train_data
test_data
raw_datasets

2022-05-23 03:48:41.830174: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 03:48:41.830235: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


                                                   text  hatespeech
0                                   @user vendendo o cu           1
1     caralho tu vem pra mantena 1 vez no ano pra fi...           0
2     18 perguntas, 18 nomes\n\n1- não tem\n2- vinic...           0
3                                       @user seu verme           1
4        @user o cara é gordo,espera oq dele??? kkkkkkk           1
...                                                 ...         ...
5683  @user sua servidão e submissão a bolsonaro é t...           0
5684  [3/8 12:44] wender: Quem liga pra roupa, comen...           1
5685                puta merda pedro não para de chorar           1
5686  Temos que defender quem nos defendeu com tanto...           0
5687     puta q pariu q cólica do caralho\nútero tóxico           0

[5688 rows x 2 columns]


DatasetDict({
    train: Dataset({
        features: ['text', 'hatespeech'],
        num_rows: 5688
    })
    test: Dataset({
        features: ['text', 'hatespeech'],
        num_rows: 2438
    })
})

In [6]:
# Tratamento básico dos dados

train_texts = raw_datasets['train'][text]
train_labels = raw_datasets['train'][label]
test_texts = raw_datasets['test'][text]
test_labels = raw_datasets['test'][label]

print("TrainTexts Length: ", len(train_texts))
print("TrainLabels Length: ", len(train_labels))
print("TestTexts Length: ", len(test_texts))
print("TestLabels Length: ", len(test_labels))

TrainTexts Length:  5688
TrainLabels Length:  5688
TestTexts Length:  2438
TestLabels Length:  2438


In [7]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1, stratify=train_labels)

In [8]:
print("TrainTexts Length: ", len(train_texts))
print("TrainLabels Length: ", len(train_labels))
print("ValidationTexts Length: ", len(val_texts))
print("ValidationLabels Length: ", len(val_labels))

TrainTexts Length:  5119
TrainLabels Length:  5119
ValidationTexts Length:  569
ValidationLabels Length:  569


In [9]:
print(sum(train_labels))
print(len(train_labels) - sum(train_labels))
print(sum(val_labels))
print(len(val_labels) - sum(val_labels))

2559
2560
285
284


In [10]:
print(test_labels[:15])

[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1]


In [21]:
# Importando o Tokenizer

from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
print(tokenizer)

Could not locate the tokenizer configuration file, will try to use the model config instead.
404 Client Error: Not Found for url: https://huggingface.co/bert-base-multilingual-casedd/resolve/main/config.json


OSError: Can't load config for 'bert-base-multilingual-casedd'. Make sure that:

- 'bert-base-multilingual-casedd' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'bert-base-multilingual-casedd' is the correct path to a directory containing a config.json file



In [12]:
# Tokenização dos datasets

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# print(val_encodings)

In [13]:
# Voltando os datasets tokenizados para instâncias da classe de Dataset

import torch

class HSDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HSDataset(train_encodings, train_labels)
val_dataset = HSDataset(val_encodings, val_labels)
test_dataset = HSDataset(test_encodings, test_labels)

In [14]:
# print(train_dataset.labels)

In [15]:
from transformers import AutoModelForSequenceClassification

# model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset
)

In [18]:
import numpy as np
from datasets import load_metric

metric_acc = load_metric("accuracy")
metric_f1 = load_metric("f1")
metric_precision = load_metric("precision")
metric_recall = load_metric("recall")

def cma(labels, predictions):
    cm = {'true_positive': 0, 'true_negative': 0, 'false_positive': 0, 'false_negative': 0}
    for i in range(0, len(labels)):
        if labels[i] == predictions[i]:
            if labels[i] == 1:
                cm['true_positive'] += 1
            else:
                cm['true_negative'] += 1
        else:
            if labels[i] == 0:
                cm['false_positive'] += 1
            else:
                cm['false_negative'] += 1
    return cm

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    result = {
        'accuracy': metric_acc.compute(predictions=predictions, references=labels),
        'precision': metric_precision.compute(predictions=predictions, references=labels),
        'recall': metric_recall.compute(predictions=predictions, references=labels),
        'f1': metric_f1.compute(predictions=predictions, references=labels),
        'total': len(predictions),
        'cm': cma(labels, predictions)
    }
    
    return result

In [19]:
# Fine-tuning with HuggingFace
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()
trainer.evaluate(eval_dataset=test_dataset)

***** Running training *****
  Num examples = 5119
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 6400


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Total,Cm
1,0.703500,0.694228,{'accuracy': 0.4991212653778559},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0},569,"{'true_positive': 0, 'true_negative': 284, 'false_positive': 0, 'false_negative': 285}"
2,0.706300,0.693686,{'accuracy': 0.4991212653778559},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0},569,"{'true_positive': 0, 'true_negative': 284, 'false_positive': 0, 'false_negative': 285}"
3,0.704000,0.697387,{'accuracy': 0.4991212653778559},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0},569,"{'true_positive': 0, 'true_negative': 284, 'false_positive': 0, 'false_negative': 285}"
4,0.703100,0.695274,{'accuracy': 0.4991212653778559},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0},569,"{'true_positive': 0, 'true_negative': 284, 'false_positive': 0, 'false_negative': 285}"
5,0.702600,0.693146,{'accuracy': 0.5008787346221442},{'precision': 0.5008787346221442},{'recall': 1.0},{'f1': 0.6674473067915692},569,"{'true_positive': 285, 'true_negative': 0, 'false_positive': 284, 'false_negative': 0}"


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 569
  Batch size = 4
/home/arthurn/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'accuracy': 0.4991212653778559}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting t

Configuration saved in test_trainer/checkpoint-6000/config.json
Model weights saved in test_trainer/checkpoint-6000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 569
  Batch size = 4
Trainer is attempting to log a value of "{'accuracy': 0.5008787346221442}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5008787346221442}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 1.0}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.6674473067915692}" of type <class 'dict'> for key "eval/f1" as a scalar

Trainer is attempting to log a value of "{'accuracy': 0.5}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 1.0}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.6666666666666666}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'true_positive': 1219, 'true_negative': 0, 'false_positive': 1219, 'false_negative': 0}" of t

{'eval_loss': 0.6931476593017578,
 'eval_accuracy': {'accuracy': 0.5},
 'eval_precision': {'precision': 0.5},
 'eval_recall': {'recall': 1.0},
 'eval_f1': {'f1': 0.6666666666666666},
 'eval_total': 2438,
 'eval_cm': {'true_positive': 1219,
  'true_negative': 0,
  'false_positive': 1219,
  'false_negative': 0},
 'eval_runtime': 204.097,
 'eval_samples_per_second': 11.945,
 'eval_steps_per_second': 2.989,
 'epoch': 5.0}